In [1]:
library(stringr)
library(parallel)

In [25]:
samples = readLines("../data/RNAseq/sample_list")

In [26]:
samples = readLines("../data/RNAseq/sample_list_2")

 merge samples from two different runs in one  

In [49]:
# setwd("/nfs/lab/projects/islet_cytok/data/RNAseq/")
# for (samp in samples[10]){
    
# system(paste( 'cat', paste0(samp,'_R1_001.fastq.gz'), paste0(gsub("L001", "L007", samp),'_R1_001.fastq.gz'),
#      ">", paste0(gsub("L001", "combined", samp),'_R1_001.fastq.gz')) )
# system(paste( 'cat', paste0(samp,'_R2_001.fastq.gz'), paste0(gsub("L001", "L007", samp),'_R2_001.fastq.gz'),
#      ">", paste0(gsub("L001", "combined", samp),'_R2_001.fastq.gz')) )    
    
# }

In [50]:
samples =gsub("L001", "combined", samples)

In [2]:
home = '/nfs/lab/projects/islet_cytok/pipeline/RNAseq/sample'

In [8]:
align = function (samp) {
outdir = paste( home, samp , "alignment", sep="/")
c1     = paste( "mkdir -p",  outdir)
c2     = paste( "cd"      ,  outdir)

r1     = paste('/nfs/lab/projects/islet_cytok/data/RNAseq/', samp, '_R1_001.fastq.gz', sep="")
r2     = paste('/nfs/lab/projects/islet_cytok/data/RNAseq/', samp, '_R2_001.fastq.gz', sep="")

pars   = paste(" LB:", samp, " SM:", samp, sep="") 

c3 = paste(
'STAR' ,
	'--runThreadN 16' ,
#'--genomeDir /nfs/lab/star_index_hg19_sorted_20151123' ,
    '--genomeDir /home/ysun/reference/starIndex/hg19' ,
	'--genomeLoad LoadAndKeep' , ###LoadAndRemove
	'--readFilesCommand zcat' ,
	'--readFilesIn', r1, r2, 
	'--outSAMattributes All' ,
	'--outSAMunmapped Within' ,
	'--outSAMattrRGline ID:1 PL:ILLUMINA',  pars,
	'--outFilterMultimapNmax 20' ,
	'--outFilterMismatchNmax 999' ,
	'--alignIntronMin 20' ,
	'--alignIntronMax 1000000' ,
	'--alignMatesGapMax 1000000' ,
	'--outSAMtype BAM Unsorted' ,
	'--quantMode TranscriptomeSAM')

system(c1)
system(c2)
system(c3)

outname = paste(outdir, samp, sep="/")    
files = c('Aligned.out.bam', 'Log.out', 'Log.final.out', 'Log.progress.out', 
          'SJ.out.tab', 'Aligned.toTranscriptome.out.bam') 
    
    
for (f in files){
c = paste("mv ", f, " ", outname, "_", f, sep="")
    system(c)
}
    
    
system("rm -r _STARtmp")
    }

In [59]:
for (s in samples){
    align(samp=s)
}